In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-big-data-bowl-2025/players.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_7.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_9.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_6.csv
/kaggle/input/nfl-big-data-bowl-2025/games.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_8.csv
/kaggle/input/nfl-big-data-bowl-2025/player_play.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_4.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_3.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_5.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_1.csv
/kaggle/input/nfl-big-data-bowl-2025/plays.csv
/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv


## <center>First things first, gotta load in the data and I want to explore the data for Amon-Ra St. Brown first and then try to extrapolate from there</center>

In [2]:
games = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/games.csv")
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/players.csv')
player_play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/player_play.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2025/plays.csv')
tracking_week_2 = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2025/tracking_week_2.csv")
players[players['displayName'] == 'Amon-Ra St. Brown']

,nflId,height,weight,birthDate,collegeName,position,displayName
1328,53541,6-1,195,NaN,Southern California,WR,Amon-Ra St. Brown


## <center>Here are all the pass receptions and rushes that St.Brown had in the game against the Commanders</center>

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
player_play[(player_play['nflId'] == 53541) & 
            ((player_play['hadRushAttempt'] == 1.0) | (player_play['hadPassReception'] == 1.0)) & 
            (player_play['gameId'] == 2022091802)]

,gameId,playId,nflId,teamAbbr,hadRushAttempt,rushingYards,hadDropback,passingYards,sackYardsAsOffense,hadPassReception,receivingYards,wasTargettedReceiver,yardageGainedAfterTheCatch,fumbles,fumbleLost,fumbleOutOfBounds,assistedTackle,forcedFumbleAsDefense,halfSackYardsAsDefense,passDefensed,quarterbackHit,sackYardsAsDefense,safetyAsDefense,soloTackle,tackleAssist,tackleForALoss,tackleForALossYardage,hadInterception,interceptionYards,fumbleRecoveries,fumbleRecoveryYards,penaltyYards,penaltyNames,wasInitialPassRusher,causedPressure,timeToPressureAsPassRusher,getOffTimeAsPassRusher,inMotionAtBallSnap,shiftSinceLineset,motionSinceLineset,wasRunningRoute,routeRan,blockedPlayerNFLId1,blockedPlayerNFLId2,blockedPlayerNFLId3,pressureAllowedAsBlocker,timeToPressureAllowedAsBlocker,pff_defensiveCoverageAssignment,pff_primaryDefensiveCoverageMatchupNflId,pff_secondaryDefensiveCoverageMatchupNflId
50940,2022091802,56,53541,DET,0,0,0,0,0,1,3,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,SCREEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51116,2022091802,316,53541,DET,0,0,0,0,0,1,49,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,True,True,1.0,IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51182,2022091802,390,53541,DET,0,0,0,0,0,1,6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,HITCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51468,2022091802,817,53541,DET,0,0,0,0,0,1,13,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,GO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51798,2022091802,1303,53541,DET,0,0,0,0,0,1,11,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,SLANT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52038,2022091802,1646,53541,DET,0,0,0,0,0,1,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,True,True,1.0,HITCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52546,2022091802,2401,53541,DET,0,0,0,0,0,1,11,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52744,2022091802,2710,53541,DET,1,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53162,2022091802,3262,53541,DET,0,0,0,0,0,1,8,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,True,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53206,2022091802,3310,53541,DET,0,0,0,0,0,1,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,False,NaN,NaN,False,False,False,1.0,OUT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## <center>Here is the specific play that I decided to use to test Plotly</center>

In [4]:
plays[(plays['playId'] == 2710) & (plays['gameId'] == 2022091802)]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
6533,2022091802,2710,(2:58) A.St. Brown right end pushed ob at WAS 17 for 58 yards (B.St-Juste).,3,1,10,DET,WAS,DET,25,02:58,22,15,N,35,0.797227,0.202773,1.226062,SINGLEBACK,3x1,12.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,OUTSIDE_RIGHT,NaN,58,58,0.070197,-0.070197,3.804456,False,TRICK,NaN,0,Cover-3,Zone


In [5]:
full_play = tracking_week_2[(tracking_week_2["gameId"] == 2022091802) & (tracking_week_2["playId"] == 316)]
full_play

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
4962963,2022091802,316,42488.0,Bobby McCain,1,BEFORE_SNAP,2022-09-18 17:12:34.6,20.0,WAS,right,43.450000,27.430000,2.59,0.45,0.26,279.21,138.80,NaN
4962964,2022091802,316,42488.0,Bobby McCain,2,BEFORE_SNAP,2022-09-18 17:12:34.7,20.0,WAS,right,43.630000,27.230000,2.61,0.38,0.26,285.62,137.71,NaN
4962965,2022091802,316,42488.0,Bobby McCain,3,BEFORE_SNAP,2022-09-18 17:12:34.8,20.0,WAS,right,43.810000,27.040000,2.65,0.38,0.26,290.72,136.80,NaN
4962966,2022091802,316,42488.0,Bobby McCain,4,BEFORE_SNAP,2022-09-18 17:12:34.9,20.0,WAS,right,43.990000,26.850000,2.67,0.49,0.27,298.06,135.41,NaN
4962967,2022091802,316,42488.0,Bobby McCain,5,BEFORE_SNAP,2022-09-18 17:12:35,20.0,WAS,right,44.180000,26.660000,2.67,0.45,0.27,303.54,134.71,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4969053,2022091802,316,NaN,football,261,AFTER_SNAP,2022-09-18 17:13:00.6,NaN,football,right,87.300003,54.619999,9.09,2.09,0.92,NaN,NaN,out_of_bounds
4969054,2022091802,316,NaN,football,262,AFTER_SNAP,2022-09-18 17:13:00.7,NaN,football,right,88.139999,54.959999,9.00,2.07,0.91,NaN,NaN,NaN
4969055,2022091802,316,NaN,football,263,AFTER_SNAP,2022-09-18 17:13:00.8,NaN,football,right,88.970001,55.310001,8.85,2.16,0.90,NaN,NaN,NaN
4969056,2022091802,316,NaN,football,264,AFTER_SNAP,2022-09-18 17:13:00.9,NaN,football,right,89.779999,55.650002,8.68,2.23,0.88,NaN,NaN,NaN


## <center>Here is my attempt at a plotly graph object, these things get crazy complex, but it seems to work well so far</center>

In [6]:
import plotly.graph_objects as go


fig_dict = {
    'data': [],
    'layout': {},
    'frames': []
}

fig_dict['layout']['xaxis'] = {
    'range': [0, 100],
    'title': {'text': 'x', 'font': {'color': 'white'}},  
    'linecolor': 'black',  
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': True 
}

fig_dict['layout']['yaxis'] = {
    'range': [0, 50],
    'title': {'text': 'y', 'font': {'color': 'white'}},
    'linecolor': 'black', 
    'tickcolor': 'black',  
    'dtick': 10, 
    'showgrid': False  
}

fig_dict['layout']['hovermode'] = 'closest'
fig_dict['layout']['font'] = {'color': 'white'} 
fig_dict['layout']['updatemenus'] = [
    {
        "buttons": [
            {
                'args': [None, {'frame': {'duration': 50, 'redraw': False},
                                'fromcurrent': True, 'transition': {'duration': 1000}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False},
                                  'mode': 'immediate',
                                  'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]


sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20, 'color': 'black'},
        'prefix': 'Time:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300},
    'pad': {'b': 10, 't': 50},
    'len': .9,
    'x': .1,
    'y': 0,
    'steps': []
}

for club in full_play['club'].unique():
    data_dict = {
        'x': full_play.loc[full_play['club'] == club, 'x'],
        'y': full_play.loc[full_play['club'] == club, 'y'],
        'mode': 'markers',
        'text': full_play.loc[full_play['club'] == club, 'displayName'],
        'name': club
    }
    fig_dict['data'].append(data_dict)

initial_frame = {'data': [], 'name': 'Initial'}
fig_dict['frames'].append(initial_frame)

for frame in full_play['frameId'].unique():
    f = {'data': [], 'name': str(frame)}
    
    for club in full_play['club'].unique():
        frame_data = {
            'x': full_play.loc[full_play['frameId'] == frame, 'x'][full_play['club'] == club],
            'y': full_play.loc[full_play['frameId'] == frame, 'y'][full_play['club'] == club],
            'mode': 'markers',
            'text': full_play.loc[full_play['frameId'] == frame, 'displayName'][full_play['club'] == club],
            'name': club
        }
        f['data'].append(frame_data) 

    fig_dict['frames'].append(f)
    
    slider_step = {
        'args': [
            [frame],
            {'frame': {'duration': 0, 'redraw': False},
             'mode': 'immediate',
             'transition': {'duration': 0}}
        ],
        'label': str(frame),
        'method': 'animate'
    }
    sliders_dict['steps'].append(slider_step) 

fig_dict['layout']['sliders'] = [sliders_dict]

fig = go.Figure(fig_dict)

if len(fig_dict['frames']) > 1:
    fig.update(data=fig_dict['frames'][1]['data']) 

fig.update_layout(paper_bgcolor='#242424', plot_bgcolor='#0a4200')
    
fig.show()


------------

# <center>WIP</center>

In [7]:
wsh_game = player_play[(player_play['nflId'] == 53541) & 
            ((player_play['hadRushAttempt'] == 1.0) | (player_play['hadPassReception'] == 1.0)) & 
            (player_play['gameId'] == 2022091802)]['playId'].to_numpy()
wsh_game = wsh_game.tolist()
wsh_game

[56, 316, 390, 817, 1303, 1646, 2401, 2710, 3262, 3310, 3800]

In [8]:
amon_ra_plays = tracking_week_2[tracking_week_2['playId'].isin(wsh_game)]
amon_ra_plays

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
765440,2022091812,56,29851.0,Aaron Rodgers,1,BEFORE_SNAP,2022-09-19 00:23:09.1,12.0,GB,left,90.270000,29.77,0.13,0.12,0.01,277.00,78.33,huddle_break_offense
765441,2022091812,56,29851.0,Aaron Rodgers,2,BEFORE_SNAP,2022-09-19 00:23:09.2,12.0,GB,left,90.280000,29.80,0.13,0.12,0.03,276.40,62.67,NaN
765442,2022091812,56,29851.0,Aaron Rodgers,3,BEFORE_SNAP,2022-09-19 00:23:09.3,12.0,GB,left,90.280000,29.80,0.11,0.11,0.01,276.40,59.33,NaN
765443,2022091812,56,29851.0,Aaron Rodgers,4,BEFORE_SNAP,2022-09-19 00:23:09.4,12.0,GB,left,90.290000,29.82,0.10,0.10,0.01,276.40,54.20,NaN
765444,2022091812,56,29851.0,Aaron Rodgers,5,BEFORE_SNAP,2022-09-19 00:23:09.5,12.0,GB,left,90.290000,29.82,0.09,0.09,0.01,276.99,50.84,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5385284,2022091802,3800,NaN,football,145,AFTER_SNAP,2022-09-18 20:00:31.6,NaN,football,left,42.349998,13.75,2.56,4.35,0.21,NaN,NaN,tackle
5385285,2022091802,3800,NaN,football,146,AFTER_SNAP,2022-09-18 20:00:31.7,NaN,football,left,42.189999,13.86,1.90,4.60,0.19,NaN,NaN,NaN
5385286,2022091802,3800,NaN,football,147,AFTER_SNAP,2022-09-18 20:00:31.8,NaN,football,left,42.049999,13.98,1.54,4.35,0.18,NaN,NaN,NaN
5385287,2022091802,3800,NaN,football,148,AFTER_SNAP,2022-09-18 20:00:31.9,NaN,football,left,41.959999,14.07,1.16,3.98,0.14,NaN,NaN,NaN
